# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [20]:
file_one = "Output/WeatherCheck.csv"
file_one_df = pd.read_csv(file_one, encoding="utf-8")
file_one_df.head()

City  Cloudiness Country        Date  Humidity  Latitude  \
0       ushuaia        90.0      AR  1586739333      87.0    -54.80   
1   kapuskasing        90.0      CA  1586739578      55.0     49.42   
2       wahiawa        75.0      US  1586739579      65.0     21.50   
3  norman wells        75.0      CA  1586739579      38.0     65.28   
4      sri aman        75.0      MY  1586739580      94.0      1.24   

   Longitude  Max Temp  Wind Speed  
0     -68.30      42.8       14.99  
1     -82.43      35.6        3.36  
2    -158.02      82.4        4.70  
3    -126.83      39.2        4.70  
4     111.46      78.8        2.19

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
gmaps.configure(api_key=g_key)

In [22]:
locations = file_one_df[["Latitude", "Longitude"]].astype(float)
humidity = file_one_df["Humidity"].astype(float)

In [23]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [24]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.


vacation_spots = file_one_df[file_one_df['Max Temp'] > 70]
vacation_spots = vacation_spots[vacation_spots['Max Temp'] < 80]
vacation_spots = vacation_spots[vacation_spots['Wind Speed'] < 10]
vacation_spots = vacation_spots[vacation_spots['Cloudiness'] == 0]
vacation_spots

City  Cloudiness Country        Date  Humidity  Latitude  \
94         warangal         0.0      IN  1586739476      44.0     18.00   
98          bapatla         0.0      IN  1586739613      85.0     15.90   
123       allapalli         0.0      IN  1586739623      27.0     19.42   
129         sonoita         0.0      MX  1586739626      36.0     31.85   
154           akyab         0.0      MM  1586739636      83.0     20.15   
160       phulabani         0.0      IN  1586739638      23.0     20.47   
171          beloha         0.0      MG  1586739642      67.0    -25.17   
173           barhi         0.0      IN  1586739643      23.0     23.90   
190      devarkonda         0.0      IN  1586739648      34.0     16.70   
221          jardim         0.0      BR  1586739660      61.0    -21.48   
259         asansol         0.0      IN  1586739673      55.0     23.68   
291          namibe         0.0      AO  1586739684      75.0    -15.20   
334  port macquarie         0.0      AU  1586739698      63.0    -31.43   
419       byron bay         0.0      AU  1586739727      60.0    -28.65   
453         sawtell         0.0      AU  1586739668      53.0    -30.38   
479          matara         0.0      LK  1586739747      89.0      5.95   
483           bilma         0.0      NE  1586739749      15.0     18.69   

     Longitude  Max Temp  Wind Speed  
94       79.58     77.94        8.77  
98       80.47     77.20        4.52  
123      80.07     79.43        3.24  
129    -112.83     71.01        8.99  
154      92.90     72.82        4.81  
160      84.23     73.81        1.16  
171      45.05     73.00        5.73  
173      80.80     75.24        1.77  
190      78.93     79.27        0.83  
221     -56.14     71.31        2.10  
259      86.98     75.54        2.91  
291      12.15     76.32        3.40  
334     152.92     73.00        4.70  
419     153.62     75.20        6.93  
453     153.10     73.40        8.05  
479      80.54     76.55        4.18  
483      12.92     79.74        7.34

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
vacation_spots["Hotel Name"] = ''
vacation_spots = vacation_spots.reset_index()
vacation_spots = vacation_spots[['City', 'Cloudiness', 'Country', 'Date', 'Humidity', 'Latitude', 'Longitude', 'Max Temp', 
                                'Wind Speed', 'Hotel Name']]
vacation_spots = pd.DataFrame(vacation_spots)
hotel_df = []
import json
vacation_spots

City  Cloudiness Country        Date  Humidity  Latitude  \
0         warangal         0.0      IN  1586739476      44.0     18.00   
1          bapatla         0.0      IN  1586739613      85.0     15.90   
2        allapalli         0.0      IN  1586739623      27.0     19.42   
3          sonoita         0.0      MX  1586739626      36.0     31.85   
4            akyab         0.0      MM  1586739636      83.0     20.15   
5        phulabani         0.0      IN  1586739638      23.0     20.47   
6           beloha         0.0      MG  1586739642      67.0    -25.17   
7            barhi         0.0      IN  1586739643      23.0     23.90   
8       devarkonda         0.0      IN  1586739648      34.0     16.70   
9           jardim         0.0      BR  1586739660      61.0    -21.48   
10         asansol         0.0      IN  1586739673      55.0     23.68   
11          namibe         0.0      AO  1586739684      75.0    -15.20   
12  port macquarie         0.0      AU  1586739698      63.0    -31.43   
13       byron bay         0.0      AU  1586739727      60.0    -28.65   
14         sawtell         0.0      AU  1586739668      53.0    -30.38   
15          matara         0.0      LK  1586739747      89.0      5.95   
16           bilma         0.0      NE  1586739749      15.0     18.69   

    Longitude  Max Temp  Wind Speed Hotel Name  
0       79.58     77.94        8.77             
1       80.47     77.20        4.52             
2       80.07     79.43        3.24             
3     -112.83     71.01        8.99             
4       92.90     72.82        4.81             
5       84.23     73.81        1.16             
6       45.05     73.00        5.73             
7       80.80     75.24        1.77             
8       78.93     79.27        0.83             
9      -56.14     71.31        2.10             
10      86.98     75.54        2.91             
11      12.15     76.32        3.40             
12     152.92     73.00        4.70             
13     153.62     75.20        6.93             
14     153.10     73.40        8.05             
15      80.54     76.55        4.18             
16      12.92     79.74        7.34

In [26]:
for index, row in vacation_spots.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    latit = row['Latitude']
    longi = row['Longitude']
    city = row['City']
    
    address = f"{latit}, {longi}"
    
    params = {
    "location": address,
    "keyword": 'hotel',
    "radius": 5000,
    "type": 'lodging',
    "key": g_key}
    
    print(f"Retrieving Results for Index {index}: {city} at {address}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"First hotel in {city}  is {results[0]['name']}.") 
        vacation_spots.loc[index, 'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")  
    print("------------")

Retrieving Results for Index 0: warangal at 18.0, 79.58.
First hotel in warangal  is Hotel Ratna.
------------
Retrieving Results for Index 1: bapatla at 15.9, 80.47.
First hotel in bapatla  is Sai Palace A/C.
------------
Retrieving Results for Index 2: allapalli at 19.42, 80.07.
First hotel in allapalli  is Hotel iK Residency/ik guest house.
------------
Retrieving Results for Index 3: sonoita at 31.85, -112.83.
First hotel in sonoita  is Hotel Excelsior.
------------
Retrieving Results for Index 4: akyab at 20.15, 92.9.
First hotel in akyab  is Yuzana Aung Hotel.
------------
Retrieving Results for Index 5: phulabani at 20.47, 84.23.
First hotel in phulabani  is Hotel KP's Salunki.
------------
Retrieving Results for Index 6: beloha at -25.17, 45.05.
Missing field/result... skipping.
------------
Retrieving Results for Index 7: barhi at 23.9, 80.8.
Missing field/result... skipping.
------------
Retrieving Results for Index 8: devarkonda at 16.7, 78.93.
First hotel in devarkonda  is 

In [43]:
narrowed_city_df = vacation_spots.rename(columns={'Latitude': 'Lat', 'Longitude': 'Lng'})
hotel_df = narrowed_city_df

In [44]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [61]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=1,
    info_box_content=[f"{info_box_template}" for index, row  in narrowed_city_df.iterrows()]
)
    
    
fig.add_layer(hotel_layer)
fig
# Display Map

Figure(layout=FigureLayout(height='420px'))